In [1]:
in_path =r"D:/rawdata/"   # use your path
out_path= "D:/cleandata/"
import numpy
import pandas as pd
pd.set_option('display.max_columns', 500)
import glob
import re
from nltk import stem
porter = stem.porter.PorterStemmer()
#from gensim import corpora, models, similarities
#import matplotlib.pyplot as plt
#%matplotlib inlineT

In [2]:
allFiles = glob.glob(in_path + "/*.txt")  # just to merge the data
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df_ = pd.read_table(file_,index_col=False, header=0)
    list_.append(df_)
    df = pd.concat(list_)
#df=df[~df.AB.isnull()]
#df=df[(df.DT!='Correction') & (df.DT!='Editorial Material')]
#df.DE.fillna(' ',inplace=True)
#df=df[df.PY<2016]
df=df[df.PY>=2000]
#df.reset_index(inplace=True, drop=True)
#df['id']=df.index
df=df.dropna(subset=["UT"])  # dropping data without a WUS number
df = df.reset_index(drop=True)
df.to_csv(out_path+"articles.csv")
#print len(df)
#node=df[['id','TI','DE','PY','SO','TC']]
#node.rename(columns={'TI':'label'},inplace=True)
#node.to_csv('../csv/nodes.csv',index=False)

In [3]:
for i in range (0, len(df.index)):  #lower-casing all references
    df.loc[i,"CR"]=str(df.loc[i,"CR"]).lower()
    df.loc[i,"SO"]=str(df.loc[i,"SO"]).lower()
    #df.loc[i,"PY"]=int(df.loc[i,"PY"]) # changing published year format to INT
print (df['SO'].count())   
df['SO'] = df['SO'].map({'mis quarterly': 'MISQ', 'information systems research': 'ISR', 'journal of management information systems':'JMIS',
                        'journal of information technology':'JIT', "information systems journal":"ISJ",
                        'journal of strategic information systems':'JSIS','european journal of information systems':'EJIS',
                        'journal of the association for information systems':'JAIS'})
df["CR"].fillna("NaN", inplace=True) # to avoid float error on ref_count string
#df=df.dropna(subset=["SO"])
print (df['SO'].count())

4899
4899


In [4]:
def ref_count(str):
    #counting total references
    ref= [x.strip() for x in str.split(';')] 
    total_reference= len(ref)
    #counting references to the 8
    counts = {"eur j inform syst": 0, "european journal of information systems": 0,
              "mis q":0, "mis quart":0, "mis quarterly" : 0, "management information systems quarterly":0,
              "j manage inform syst":0,"journal of management information systems":0,
              "inform syst res":0,
              "j inf technol":0, "j inform technol":0, "journal of information technology":0,
              "j assoc inf syst":0,
              "inform syst j":0,
              "j strategic inf syst":0, "journal of strategic information systems":0}
    words = [x.strip() for x in str.split(',')]
    for word in words:
        if word in counts:
            counts[word] += 1
    return (counts, total_reference)

In [5]:
def merge_journals(dict): # merging different abrevations of same journal
    dict["MISQ"] = dict["mis q"]+dict["mis quart"] + dict["mis quarterly"] + dict["management information systems quarterly"]
    dict["JIT"]= dict["j inf technol"] + dict ["j inform technol"] + dict["journal of information technology"]
    dict["EJIS"]= dict["eur j inform syst"] + dict["european journal of information systems"]
    dict["JMIS"]= dict["j manage inform syst"] + dict["journal of management information systems"]
    dict["ISR"] = dict["inform syst res"]
    dict["JAIS"]=dict["j assoc inf syst"]
    dict["ISJ"]= dict["inform syst j"]
    dict["JSIS"]=dict["j strategic inf syst"] + dict["journal of strategic information systems"]
    
    return dict

In [6]:
dic=[]  # article id, source journal, counts of citation to each one of 8 journals, total number of references of the article, year of publication
for i in range (0, len(df.index)):
    b = ref_count(df.loc[i,"CR"])
    a = merge_journals(b[0])
    g= {'id': i,'source': df.loc[i,"SO"],'target':[{'j':'MISQ','count':a["MISQ"]},{'j':'JIT','count':a["JIT"]},
                                              {'j':'EJIS','count':a["EJIS"]},{'j':'JMIS','count':a["JMIS"]},
                                              {'j':'ISR','count':a["ISR"]},{'j':'JAIS','count':a["JAIS"]},
                                              {'j':'ISJ','count':a["ISJ"]},{'j':'JSIS','count':a["JSIS"]}],'total_ref': b[1], 'year': df.loc[i,"PY"] }
    dic.append(g)

In [7]:
total=0 #total number of references of all articles
for article in dic:
    total +=article['total_ref']
total

305881

In [8]:
final={} # initializing final
for i in range (2000,2019):
    final[i]={'ISJ':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JSIS':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'ISR':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'MISQ':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JAIS' :{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       "EJIS":{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       'JIT':{'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0},
       "JMIS": {'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0}}

In [9]:
#each journal number of citations to other journals based on year
listofj=['MISQ', 'ISR', 'JMIS','JIT','ISJ', "JSIS", 'EJIS', 'JAIS' ] # Cited article is in these journals
listofso=['MISQ', 'ISR', 'JMIS','JIT','ISJ', "JSIS", 'EJIS', 'JAIS' ] # The article is in these journals
                        
for article in dic:
    year=article['year']
    source=article['source']
    targets=article['target']
    for target in targets:
        jname=target['j']
        jcount=target['count']
        for so in listofso:
            for ta in listofj:
                for i in range (2000,2019): # We have articles from 2000 to 2018
                    if source==so:
                        if jname==ta:
                            if year==i:
                                final[i][ta][so]=final[i][ta][so]+jcount  # rows will be the source of article, columns are cited journals

In [10]:
citation=pd.DataFrame(data=final[2001])
citation

,EJIS,ISJ,ISR,JAIS,JIT,JMIS,JSIS,MISQ
EJIS,19,7,7,0,19,7,2,10
ISJ,10,2,10,0,2,10,2,25
ISR,5,2,51,0,1,23,1,50
JAIS,0,0,0,0,0,0,0,0
JIT,7,6,10,0,7,7,3,21
JMIS,1,0,45,0,1,57,2,100
JSIS,5,2,17,0,2,13,6,52
MISQ,4,2,53,0,1,26,0,97


In [11]:
a="EJIS" # the source journal we want the table of citation over time
total_so = 0
overtime_change_so={} 
overtime_change_so_per={} # percentage of citing 8 journals over one year
overtime_change_total_so_per={} #percentage of citing 8 journals over all years
for i in range (2000,2019): # initializing overtimechange
    overtime_change_so[i]={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0,"Sum":0}
    overtime_change_so_per[i]={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0,"Sum":0}
    overtime_change_total_so_per[i]={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0,"Sum":0}
for i in range (2000,2019):
    for ta in listofj:
        overtime_change_so[i][ta]=final[i][ta][a]
        overtime_change_so[i]["Sum"]=overtime_change_so[i]["Sum"]+overtime_change_so[i][ta]
    for ta in listofj:
        if overtime_change_so[i]["Sum"] == 0:
            overtime_change_so_per[i][ta]=0
        else:
            overtime_change_so_per[i][ta]= overtime_change_so[i][ta]/overtime_change_so[i]["Sum"]
            overtime_change_so_per[i]["Sum"]=overtime_change_so_per[i]["Sum"]+overtime_change_so_per[i][ta]
    total_so= total_so + overtime_change_so[i]["Sum"]
for i in range (2000,2019):
    for ta in listofj:
        #if total_so ==0:
         #   overtime_change_total_so_per[i][ta]=0
        #else:
        overtime_change_total_so_per[i][ta] = overtime_change_so[i][ta] / total_so
        overtime_change_total_so_per[i]["Sum"]=overtime_change_total_so_per[i]["Sum"]+overtime_change_total_so_per[i][ta]

In [12]:
a="EJIS" # the cited journal we want the table of citation over time
total_ta = 0
overtime_change_ta={} 
overtime_change_ta_per={}
overtime_change_total_ta_per={}
for i in range (2000,2019): # initializing overtimechange
    overtime_change_ta[i]={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0,"Sum":0}
    overtime_change_ta_per[i]={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0,"Sum":0}
    overtime_change_total_ta_per[i]={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0,"Sum":0}
for i in range (2000,2019):
    for so in listofso:
        overtime_change_ta[i][so]=final[i][a][so]
        overtime_change_ta[i]["Sum"]=overtime_change_ta[i]["Sum"]+overtime_change_ta[i][so] 
    for so in listofso:
        if overtime_change_ta[i]["Sum"] == 0:
            overtime_change_ta_per[i][so]=0
        else:
            overtime_change_ta_per[i][so]= overtime_change_ta[i][so]/overtime_change_ta[i]["Sum"]
            overtime_change_ta_per[i]["Sum"]=overtime_change_ta_per[i]["Sum"]+overtime_change_ta_per[i][so]
    total_ta = total_ta + overtime_change_ta[i]["Sum"]
for i in range (2000,2019):
    for so in listofso:
        overtime_change_total_ta_per[i][so] = overtime_change_ta[i][so] / total_ta
        overtime_change_total_ta_per[i]["Sum"]=overtime_change_total_ta_per[i]["Sum"]+overtime_change_total_ta_per[i][so]

In [13]:
print(total_so,total_ta)

8729 4142


In [14]:
df_so = pd.DataFrame(data=overtime_change_so) # rows= cited journals from the source journal's article
df_so_per = pd.DataFrame(data=overtime_change_so_per)
df_so_total_per = pd.DataFrame(data=overtime_change_total_so_per)
df_ta = pd.DataFrame(data=overtime_change_ta) # rows= source journals which cited the target journal's article
df_ta_per = pd.DataFrame(data=overtime_change_ta_per)
df_ta_total_per = pd.DataFrame(data=overtime_change_total_ta_per)

In [15]:
writer = pd.ExcelWriter('EJIS.xlsx')
df_so.to_excel(writer,'So')
df_so_per.to_excel(writer,'So_per')
df_so_total_per.to_excel(writer,'So_total_per')
df_ta.to_excel(writer,'Ta')
df_ta_per.to_excel(writer,'Ta_per')
df_ta_total_per.to_excel(writer,'Ta_total_per')
writer.save()

In [18]:
#Articles per journals per year, need to make the table in excel by hand!
article_per_year = []
b = df.groupby(["SO","PY"])
for so in listofso:
    g=so
    article_per_year.append(g)
    for i in range(2000,2019):
        if (so, i) not in b.groups:
            g=0
            article_per_year.append(g)
        else:    
            g = len(b.groups[(so,i)])
            article_per_year.append(g)

In [16]:
# Given citation to basket of eight per journal per year:
basket_citation_per_year={} 
basket_citation_per_year_percent={}
for i in range (2000,2019): # initializing
    basket_citation_per_year[i]={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0,"Sum":0}
    basket_citation_per_year_percent[i]={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0,"Sum":0}
for i in range (2000,2019):
    for so in listofso:
        for ta in listofj: 
            basket_citation_per_year[i][so]=final[i][ta][so]+basket_citation_per_year[i][so]
        basket_citation_per_year[i]["Sum"]=basket_citation_per_year[i]["Sum"]+basket_citation_per_year[i][so]
    for so in listofso:
        basket_citation_per_year_percent[i][so]= basket_citation_per_year[i][so]/basket_citation_per_year[i]["Sum"]
        basket_citation_per_year_percent[i]["Sum"]=basket_citation_per_year_percent[i]["Sum"]+basket_citation_per_year_percent[i][so]

In [17]:
#  number of times cited by basket of eight per journal per year:
basket_citation_per_year_received={} 
basket_citation_per_year_received_percent={}
for i in range (2000,2019): # initializing
    basket_citation_per_year_received[i]={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0,"Sum":0}
    basket_citation_per_year_received_percent[i]={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0,"Sum":0}
for i in range (2000,2019):
    for ta in listofj:
        for so in listofso: 
            basket_citation_per_year_received[i][ta]=final[i][ta][so]+basket_citation_per_year_received[i][ta]
        basket_citation_per_year_received[i]["Sum"]=basket_citation_per_year_received[i]["Sum"]+basket_citation_per_year_received[i][ta]
    for ta in listofj:
        basket_citation_per_year_received_percent[i][ta]= basket_citation_per_year_received[i][ta]/basket_citation_per_year_received[i]["Sum"]
        basket_citation_per_year_received_percent[i]["Sum"]=basket_citation_per_year_received_percent[i]["Sum"]+basket_citation_per_year_received_percent[i][ta]

In [20]:
basket_given = pd.DataFrame(data=basket_citation_per_year)
basket_given_per = pd.DataFrame(data=basket_citation_per_year_percent)
basket_received = pd.DataFrame(data=basket_citation_per_year_received)
basket_received_per = pd.DataFrame(data=basket_citation_per_year_received_percent)

In [21]:
writer = pd.ExcelWriter('total citation per year.xlsx')
basket_given.to_excel(writer,'given')
basket_given_per.to_excel(writer,'given_per')
basket_received.to_excel(writer,'received')
basket_received_per.to_excel(writer,'received_per')
writer.save()

In [21]:
#Total number of cication of each journal per year (these citations are to all sources)
listofso=['MISQ', 'ISR', 'JMIS','JIT','ISJ', "JSIS", 'EJIS', 'JAIS' ] # The article is in these journals
total_citation_per_year={} # initializing
for i in range (2000,2019):
    total_citation_per_year[i]={'MISQ':0, 'ISR':0, 'JIT':0, 'EJIS':0, 'JAIS':0, 'ISJ':0, "JSIS":0,'JMIS':0}
                        
for article in dic:
    year=article['year']
    source=article['source']
    ref=article['total_ref']
    for so in listofso:
            for i in range (2000,2019): # We have articles from 2000 to 2018
                if source==so:
                    if year==i:
                        total_citation_per_year[i][so]=total_citation_per_year[i][so]+ref  # total references of journals per year

In [22]:
pd.DataFrame(data=total_citation_per_year) # rows= cited journals from the source journal's article

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
EJIS,877,870,1258,1200,1011,1942,2636,3070,3259,2840,3010,3086,3055,3360,3463,3252,2746,3265,632
ISJ,968,969,800,879,790,1055,876,1308,1761,1631,1500,1569,1248,1466,1584,1893,1794,2309,1327
ISR,1268,1230,1598,1022,1134,1030,1351,1445,1679,1946,3041,2813,4578,4467,3368,3219,3736,3174,910
JAIS,0,0,0,0,0,0,1941,1983,2069,2459,3193,2442,3252,2547,2850,3289,2408,2634,736
JIT,986,863,1099,984,856,1261,1403,1427,1552,1778,1986,1481,1700,1749,1712,2010,2003,2049,363
JMIS,1999,1874,2023,2180,2149,2754,2745,2430,2445,2541,2748,3163,2930,2581,3011,3860,2819,3101,744
JSIS,626,822,851,565,1160,909,951,1313,1065,997,1870,1922,1928,1720,1675,1574,1910,2067,0
MISQ,1489,1497,1158,1797,1780,2264,3180,2959,2954,3064,3676,4425,4868,5827,5099,4638,3758,5615,1178


In [12]:
a

{'EJIS': 3,
 'ISJ': 4,
 'ISR': 0,
 'JAIS': 0,
 'JIT': 0,
 'JMIS': 1,
 'JSIS': 2,
 'MISQ': 2,
 'eur j inform syst': 3,
 'european journal of information systems': 0,
 'inform syst j': 4,
 'inform syst res': 0,
 'j assoc inf syst': 0,
 'j inf technol': 0,
 'j inform technol': 0,
 'j manage inform syst': 1,
 'j strategic inf syst': 2,
 'journal of information technology': 0,
 'journal of management information systems': 0,
 'journal of strategic information systems': 0,
 'management information systems quarterly': 0,
 'mis q': 0,
 'mis quart': 2,
 'mis quarterly': 0}

In [13]:
final

{2000: {'EJIS': {'EJIS': 15,
   'ISJ': 15,
   'ISR': 0,
   'JAIS': 0,
   'JIT': 10,
   'JMIS': 3,
   'JSIS': 4,
   'MISQ': 4},
  'ISJ': {'EJIS': 4,
   'ISJ': 11,
   'ISR': 0,
   'JAIS': 0,
   'JIT': 2,
   'JMIS': 2,
   'JSIS': 0,
   'MISQ': 0},
  'ISR': {'EJIS': 13,
   'ISJ': 15,
   'ISR': 43,
   'JAIS': 0,
   'JIT': 9,
   'JMIS': 72,
   'JSIS': 7,
   'MISQ': 39},
  'JAIS': {'EJIS': 0,
   'ISJ': 0,
   'ISR': 0,
   'JAIS': 0,
   'JIT': 0,
   'JMIS': 0,
   'JSIS': 0,
   'MISQ': 0},
  'JIT': {'EJIS': 3,
   'ISJ': 16,
   'ISR': 0,
   'JAIS': 0,
   'JIT': 13,
   'JMIS': 4,
   'JSIS': 4,
   'MISQ': 3},
  'JMIS': {'EJIS': 14,
   'ISJ': 16,
   'ISR': 8,
   'JAIS': 0,
   'JIT': 12,
   'JMIS': 132,
   'JSIS': 6,
   'MISQ': 35},
  'JSIS': {'EJIS': 8,
   'ISJ': 8,
   'ISR': 1,
   'JAIS': 0,
   'JIT': 4,
   'JMIS': 11,
   'JSIS': 2,
   'MISQ': 0},
  'MISQ': {'EJIS': 45,
   'ISJ': 41,
   'ISR': 59,
   'JAIS': 0,
   'JIT': 39,
   'JMIS': 142,
   'JSIS': 24,
   'MISQ': 118}},
 2001: {'EJIS': {'EJIS': 